## Install Catboost!

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 14.2 MB/s eta 0:00:00


## Benchmark with individual parts of the proposed technique

In [ ]:
#Main script for proposed model learning
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor, HistGradientBoostingRegressor, BaggingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Configuration
horizons = [0, 1, 2, 5]

features = ['Longitude','PFG_surface_pressure','Latitude','Slope','Elevation','Year','EVI']
target = 'ALT_Max'

hyperparms_ET = {0: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 20},
                 1: {'n_estimators': 700, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 50},
                 2: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 20},
                 5: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 20}}

hyperparms_Bagging = {0: {'n_estimators': 2500, 'max_samples': 1.0, 'max_features': 1.0, 'bootstrap_features': False},
                 1: {'n_estimators': 2500, 'max_samples': 1.0, 'max_features': 0.8, 'bootstrap_features': False},
                 2: {'n_estimators': 2000, 'max_samples': 1.0, 'max_features': 1.0, 'bootstrap_features': True},
                 5: {'n_estimators': 2500, 'max_samples': 1.0, 'max_features': 0.9, 'bootstrap_features': False}}

hyperparms_cat = {0: {'bagging_temperature': 5, 'random_strength': 1, 'depth': 6, 'learning_rate': 0.1, 'l2_leaf_reg': 7, 'iterations': 1500},
                 1: {'bagging_temperature': 1, 'random_strength': 1, 'depth': 4, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 2500},
                 2: {'bagging_temperature': 1, 'random_strength': 1, 'depth': 6, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 2500},
                 5: {'bagging_temperature': 1, 'random_strength': 1, 'depth': 6, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 2500}}

# Load and evaluate
for hrz in horizons:
    # Dynamically define models for the current horizon
    models = [
        ('Extra Trees', ExtraTreesRegressor(random_state=42, **hyperparms_ET[hrz])),
        ('Bagging Regressor', BaggingRegressor(random_state=42, **hyperparms_Bagging[hrz])),
        ('CatBoost', CatBoostRegressor(verbose=0, random_seed=42, **hyperparms_cat[hrz]))
    ]
    print(f'*********************** HRZ = {hrz} years')
    train_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])
    test_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])

    train_info = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv',usecols=['MainRegion', 'Region', 'Location', 'Latitude', 'Longitude', 'Year'])
    test_info = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv',usecols=['MainRegion', 'Region', 'Location', 'Latitude', 'Longitude', 'Year'])


    train_data = train_data[features + [target]]
    test_data = test_data[features + [target]]

    X_train = train_data.drop(columns=target)
    y_train = train_data[target]
    X_test = test_data.drop(columns=target)
    y_test = test_data[target]


    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Step 1: CV predictions and RMSE
    preds_test = {}
    preds_train_cv = {}
    rmse_scores = {}

    for name, model in models:
        y_pred_cv = cross_val_predict(model, X_train, y_train, cv=cv)
        model.fit(X_train, y_train)
        y_pred_test = model.predict(X_test)

        preds_test[name] = y_pred_test
        preds_train_cv[name] = y_pred_cv

        #training performance
        rmse = np.sqrt(mean_squared_error(y_train, y_pred_cv))
        r2 = r2_score(y_train, y_pred_cv)
        rmse_scores[name] = rmse

        print(f"{name}: CV Training RMSE = {rmse:.3f}, R2={r2:.3f}| Testing RMSE={np.sqrt(mean_squared_error(y_test, y_pred_test)):.3f}, R2={r2_score(y_test, y_pred_test):.3f}")


    # Step 2: Compute CV-based weights
    inverse_rmse = {k: 1 / v for k, v in rmse_scores.items()}
    total_inv = sum(inverse_rmse.values())
    weights = {k: v / total_inv for k, v in inverse_rmse.items()}
    # print("CV-based Weights:", weights)

    # Step 3: Weighted predictions
    weighted_test_pred = sum(preds_test[m] * weights[m] for m in weights)
    weighted_cv_pred = sum(preds_train_cv[m] * weights[m] for m in weights)

    # Step 4: Evaluate weighted ensemble
    rmse = np.sqrt(mean_squared_error(y_test, weighted_test_pred))
    mae = mean_absolute_error(y_test, weighted_test_pred)
    r2 = r2_score(y_test, weighted_test_pred)
    print(f"==> Weighted Ensemble - RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

    # Ensure no index issues
    train_info = train_info.reset_index(drop=True)
    test_info = test_info.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    # Build outputs with metadata
    df_train_out = train_info.copy()
    df_test_out = test_info.copy()

    df_train_out['y_train'] = y_train
    df_test_out['y_test'] = y_test

    for name in preds_train_cv:
        df_train_out[f'y_trpred_{name}'] = preds_train_cv[name]
        df_test_out[f'y_tepred_{name}'] = preds_test[name]
        df_train_out[f'weight_{name}'] = weights[name]
        df_test_out[f'weight_{name}'] = weights[name]

    df_train_out['y_trpred_ensemble'] = weighted_cv_pred
    df_train_out['weight_ensemble'] = 1.0
    df_test_out['y_tepred_ensemble'] = weighted_test_pred
    df_test_out['weight_ensemble'] = 1.0

    # df_train_out.to_csv(f'/content/drive/MyDrive/UND/ALT/ALT_hrz{hrz}_TrainPredictions.csv', index=False)
    # df_test_out.to_csv(f'/content/drive/MyDrive/UND/ALT/ALT_hrz{hrz}_TestPredictions.csv', index=False)

*********************** HRZ = 0 years
Extra Trees: CV Training RMSE = 12.595, R2=0.909| Testing RMSE=24.946, R2=0.802
Bagging Regressor: CV Training RMSE = 13.045, R2=0.902| Testing RMSE=26.362, R2=0.779
CatBoost: CV Training RMSE = 13.268, R2=0.899| Testing RMSE=25.493, R2=0.793
==> Weighted Ensemble - RMSE: 24.241, MAE: 14.627, R2: 0.813
*********************** HRZ = 1 years
Extra Trees: CV Training RMSE = 13.063, R2=0.902| Testing RMSE=24.494, R2=0.809
Bagging Regressor: CV Training RMSE = 14.206, R2=0.884| Testing RMSE=26.920, R2=0.770
CatBoost: CV Training RMSE = 13.703, R2=0.892| Testing RMSE=26.730, R2=0.773
==> Weighted Ensemble - RMSE: 24.328, MAE: 14.653, R2: 0.812
*********************** HRZ = 2 years
Extra Trees: CV Training RMSE = 12.795, R2=0.906| Testing RMSE=24.488, R2=0.809
Bagging Regressor: CV Training RMSE = 14.106, R2=0.886| Testing RMSE=27.192, R2=0.765
CatBoost: CV Training RMSE = 13.242, R2=0.899| Testing RMSE=24.779, R2=0.805
==> Weighted Ensemble - RMSE: 24.30

*********************** HRZ = 2 years
Selected features: ['Longitude', 'PFG_surface_pressure', 'Latitude', 'Slope', 'Elevation', 'Year', 'EVI']
Extra Trees: CV Training RMSE = 12.795, R2=0.906| Testing RMSE=24.488, R2=0.809
Bagging Regressor: CV Training RMSE = 14.106, R2=0.886| Testing RMSE=27.192, R2=0.765
CatBoost: CV Training RMSE = 13.242, R2=0.899| Testing RMSE=24.779, R2=0.805
==> Weighted Ensemble - RMSE: 24.308, MAE: 13.865, R2: 0.812
*********************** HRZ = 5 years
Selected features: ['Longitude', 'PFG_surface_pressure', 'Latitude', 'Slope', 'Elevation', 'Year', 'EVI']
Extra Trees: CV Training RMSE = 13.106, R2=0.901| Testing RMSE=24.654, R2=0.807
Bagging Regressor: CV Training RMSE = 13.321, R2=0.898| Testing RMSE=27.324, R2=0.763
CatBoost: CV Training RMSE = 13.995, R2=0.888| Testing RMSE=26.023, R2=0.785
==> Weighted Ensemble - RMSE: 24.574, MAE: 14.757, R2: 0.808


## Multi-Stage Feature Selection

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold, cross_val_score
from sklearn.utils import resample
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import Counter
import multiprocessing

# Setup
n_jobs = multiprocessing.cpu_count()
hrz = 0

# Load dataset
print('- Loading dataset and preprocessing...')
train_df = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])

categorical_features = ['LC_Type1', 'LC_Type2', 'LC_Type3', 'LC_Type4', 'LC_Type5']
numerical_features = [col for col in train_df.columns if col not in categorical_features + ['ALT_Max']]
all_feature_names = categorical_features + numerical_features

X = train_df.drop(columns=['ALT_Max'])
y = train_df['ALT_Max']

# Ensure categorical columns are properly typed before any resampling
X[categorical_features] = X[categorical_features].astype('category')
cat_feature_indices = [X.columns.get_loc(col) for col in categorical_features]

# Helper function to evaluate RMSE + penalty
def evaluate_features(X_subset, y, cat_idx, selected_features, cv=5, alpha=0.05):
    model = CatBoostRegressor(cat_features=cat_idx, thread_count=n_jobs, verbose=False, random_state=42)
    scores = -1 * cross_val_score(model, X_subset, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=n_jobs)
    rmse = np.mean(scores)
    penalized_score = rmse + alpha * len(selected_features)
    return rmse, penalized_score

# Step 1: Bootstrap Stability
print('- Step 1: Bootstrap stability...')
n_bootstraps = 20
stable_counter = Counter()

for seed in range(n_bootstraps):
    X_sample, y_sample = resample(X, y, random_state=seed)

    # Very important: recast categorical columns after resample
    X_sample[categorical_features] = X_sample[categorical_features].astype('category')

    model = CatBoostRegressor(cat_features=cat_feature_indices, thread_count=n_jobs, verbose=False, random_state=42)
    model.fit(X_sample, y_sample)

    sample_pool = Pool(X_sample, y_sample, cat_features=cat_feature_indices)
    shap_vals = model.get_feature_importance(sample_pool, type='ShapValues')

    sample_mean_shap = np.abs(shap_vals[:, :-1]).mean(axis=0)
    top_feats = [f for _, f in sorted(zip(sample_mean_shap, X_sample.columns), reverse=True)[:15]]
    stable_counter.update(top_feats)

stable_features = [f for f, count in stable_counter.items() if count > n_bootstraps // 2]
print(f'-- Selected features from Step 1: {stable_features}')

# Step 2–4: Floating Selection + Permutation Pruning + Nested CV
print('- Step 2-4: Floating Selection + Permutation Pruning + Nested CV...')
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
results = []
prev_score = np.inf
final_selected = []

for f in stable_features:
    if f in final_selected:
        continue
    test_subset = final_selected + [f]
    X_subset = X[test_subset]
    cat_idx = [i for i, col in enumerate(test_subset) if col in categorical_features]

    rmse, penalized = evaluate_features(X_subset, y, cat_idx, test_subset, cv=5)
    if penalized < prev_score:
        final_selected.append(f)
        prev_score = penalized
print(f'-- Selected features from Step 2-4: {final_selected}')

# Step 5: Permutation Importance Pruning
print('- Step 5: Permutation Pruning...')
X_final = X[final_selected]
final_cat_idx = [i for i, col in enumerate(X_final.columns) if col in categorical_features]

model = CatBoostRegressor(cat_features=final_cat_idx, thread_count=n_jobs, verbose=False, random_state=42)
model.fit(X_final, y)

result = permutation_importance(model, X_final, y, n_repeats=5, random_state=42, n_jobs=n_jobs)
imp_mean = result.importances_mean
final_selected = [feat for feat, imp in zip(final_selected, imp_mean) if imp > 0.001]
print(f'-- Selected features after Permutation Pruning: {final_selected}')

# Final Evaluation
final_cat_idx = [i for i, col in enumerate(final_selected) if col in categorical_features]
final_rmse, final_penalized = evaluate_features(X[final_selected], y, final_cat_idx, final_selected, cv=5)

# Report
print("\n========================================")
print(f"Final selected features ({len(final_selected)}):")
for feat in final_selected:
    print(f"- {feat}")
print(f"Final CV RMSE: {final_rmse:.4f}")
print(f"Final Penalized Score: {final_penalized:.4f}")

# Performance on Train/Test Split
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=5, shuffle=True, random_state=42)

final_selected = ['Longitude', 'PFG_surface_pressure', 'Latitude', 'Slope', 'Elevation', 'Year', 'EVI']
horizons = [0, 1, 2, 5]
for hrz in horizons:
    # Performance on Train/Test Split
    train_df = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])

    X_train = train_df[final_selected]

    y_train = train_df['ALT_Max']

    # Cross-Validation Predictions
    cv_model = CatBoostRegressor(thread_count=-1, verbose=False, random_state=42)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    cv_predictions = cross_val_predict(cv_model, X_train[final_selected], y_train, cv=cv, n_jobs=-1)

    # Compute Final CV RMSE and R²
    final_cv_rmse = np.sqrt(mean_squared_error(y_train, cv_predictions))
    final_cv_r2 = r2_score(y_train, cv_predictions)
    print(f"=================hrz={hrz}=======================")
    print(f'CatBoost Final Training: RMSE={final_cv_rmse:.4f},R2={final_cv_r2:.4f}')


- Loading dataset and preprocessing...
- Step 1: Bootstrap stability...
-- Selected features from Step 1: ['Longitude', 'PFG_surface_pressure', 'Latitude', 'Aspect', 'Slope', 'Elevation', 'u_component_of_wind_10m', 'surface_pressure', 'SFG_v_component_of_wind_10m', 'NDVI', 'SFG_surface_pressure', 'Year', 'NFG_lake_ice_depth', 'EVI']
- Step 2-4: Floating Selection + Permutation Pruning + Nested CV...
-- Selected features from Step 2-4: ['Longitude', 'PFG_surface_pressure', 'Latitude', 'Slope', 'Elevation', 'Year', 'EVI']
- Step 5: Permutation Pruning...
-- Selected features after Permutation Pruning: ['Longitude', 'PFG_surface_pressure', 'Latitude', 'Slope', 'Elevation', 'Year', 'EVI']

Final selected features (7):
- Longitude
- PFG_surface_pressure
- Latitude
- Slope
- Elevation
- Year
- EVI
Final CV RMSE: 40.9781
Final Penalized Score: 41.3281

CatBoost Final Training: RMSE=7.5058, MAE=5.5074, R2=0.9677
CatBoost Final Testing:  RMSE=25.0218, MAE=15.7130, R2=0.8011


In [ ]:
## FIND OUT WHICH COMBINATION OF MODELS TO PROPOSE
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import (
    ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
)
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, BaggingRegressor, IsolationForest
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor

# Configuration
horizons = [0, 1, 2, 5]

features = ['Longitude','PFG_surface_pressure','Latitude','Slope','Elevation','Year','EVI']

target = 'ALT_Max'

# Expanded model list
model_pool = [
    # Tree-Based Models
    ('Extra Trees', ExtraTreesRegressor(n_estimators=70, random_state=42)),
    ('Random Forest', RandomForestRegressor(n_estimators=70, random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(random_state=42)),
    ('HistGradientBoosting', HistGradientBoostingRegressor(random_state=42)),
    ('IsolationForest', IsolationForest(n_estimators=70, random_state=42)),

    # Boosted Models
    ('XGBoost', XGBRegressor(random_state=42, verbosity=0)),
    ('CatBoost', CatBoostRegressor(random_seed=42, verbose=0)),
    ('LightGBM', LGBMRegressor(random_state=42, verbose=-1)),

    # Linear Models
    ('Ridge Regression', Ridge(random_state=42)),
    ('Lasso Regression', Lasso(random_state=42)),
    ('Elastic Net', ElasticNet(random_state=42)),

    # K-Nearest Neighbors
    ('KNN Regressor', KNeighborsRegressor(n_neighbors=7)),

    # Support Vector Machines
    ('SVR RBF', SVR(kernel='rbf', C=1.0, epsilon=0.2)),
    ('SVR Linear', SVR(kernel='poly', C=1.0, epsilon=0.2)),

    # Bagging
    ('Bagging Regressor', BaggingRegressor(random_state=42)),

    # Deep Learning models (safe simple ones)
    ('MLP Small', MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)),
    ('MLP Medium', MLPRegressor(hidden_layer_sizes=(128, 64, 32), max_iter=500, random_state=42)),
    ('MLP Large', MLPRegressor(hidden_layer_sizes=(512, 256, 256, 128, 64), max_iter=500, random_state=42)),
]


cv = KFold(n_splits=5, shuffle=True, random_state=42)

for hrz in horizons:
    print(f'================ Horizon {hrz} Years ================')

    # Load datasets
    train_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])
    test_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])

    X_train = train_data[features]
    y_train = train_data[target]
    X_test = test_data[features]
    y_test = test_data[target]

    # Storage for all models
    model_predictions_cv = {}
    model_predictions_test = {}
    model_cv_rmse = {}

    for name, model in model_pool:
        try:
            y_pred_cv = cross_val_predict(model, X_train, y_train, cv=cv)
            model.fit(X_train, y_train)
            y_pred_test = model.predict(X_test)

            model_predictions_cv[name] = y_pred_cv
            model_predictions_test[name] = y_pred_test

            rmse = np.sqrt(mean_squared_error(y_train, y_pred_cv))
            model_cv_rmse[name] = rmse

            print(f"{name}: CV RMSE={rmse:.3f}, Test R2={r2_score(y_test, y_pred_test):.3f}")

        except Exception as e:
            print(f"{name} failed: {e}")

    # === Greedy Ensemble Selection ===
    selected_models = []
    best_r2 = -np.inf
    remaining_models = list(model_predictions_test.keys())

    while remaining_models:
        best_candidate = None
        best_candidate_r2 = best_r2

        for candidate in remaining_models:
            temp_models = selected_models + [candidate]
            temp_weights = {m: 1/model_cv_rmse[m] for m in temp_models}
            total_weight = sum(temp_weights.values())
            temp_weights = {m: w/total_weight for m,w in temp_weights.items()}

            weighted_pred = sum(model_predictions_test[m] * temp_weights[m] for m in temp_models)
            r2 = r2_score(y_test, weighted_pred)

            if r2 > best_candidate_r2:
                best_candidate_r2 = r2
                best_candidate = candidate

        if best_candidate and best_candidate_r2 > best_r2:
            selected_models.append(best_candidate)
            remaining_models.remove(best_candidate)
            best_r2 = best_candidate_r2
            print(f"--> Added {best_candidate} to ensemble. New Test R2 = {best_r2:.4f}")
        else:
            break

    print(f"\nFinal Ensemble Models: {selected_models}")

    # Final weighted prediction
    final_weights = {m: 1/model_cv_rmse[m] for m in selected_models}
    total_final_weight = sum(final_weights.values())
    final_weights = {m: w/total_final_weight for m,w in final_weights.items()}

    final_pred = sum(model_predictions_test[m] * final_weights[m] for m in selected_models)

    # Final Metrics
    rmse = np.sqrt(mean_squared_error(y_test, final_pred))
    mae = mean_absolute_error(y_test, final_pred)
    r2 = r2_score(y_test, final_pred)
    print(f"\nFinal Ensemble Test Performance: RMSE={rmse:.3f}, MAE={mae:.3f}, R2={r2:.3f}")


================ Horizon 0 Years ================
Extra Trees: CV RMSE=12.667, Test R2=0.785
Random Forest: CV RMSE=13.313, Test R2=0.777
Gradient Boosting: CV RMSE=18.891, Test R2=0.595
HistGradientBoosting: CV RMSE=14.935, Test R2=0.651
IsolationForest: CV RMSE=106.196, Test R2=-3.809
XGBoost: CV RMSE=13.039, Test R2=0.681
CatBoost: CV RMSE=14.206, Test R2=0.801
LightGBM: CV RMSE=14.936, Test R2=0.643
Ridge Regression: CV RMSE=38.788, Test R2=0.068
Lasso Regression: CV RMSE=38.821, Test R2=0.070
Elastic Net: CV RMSE=38.820, Test R2=0.070
KNN Regressor: CV RMSE=30.219, Test R2=0.318
SVR RBF: CV RMSE=42.357, Test R2=-0.118
SVR Linear: CV RMSE=42.510, Test R2=-0.122
Bagging Regressor: CV RMSE=13.826, Test R2=0.771
MLP Small: CV RMSE=38.782, Test R2=0.078
MLP Medium: CV RMSE=37.084, Test R2=0.103
MLP Large: CV RMSE=54.868, Test R2=-0.677
--> Added CatBoost to ensemble. New Test R2 = 0.8011
--> Added Extra Trees to ensemble. New Test R2 = 0.8111
--> Added Bagging Regressor to ensemble. Ne

## finetuning models with selected features

In [ ]:
# Main script for proposed model learning with tuning
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import RandomizedSearchCV

# Configuration
horizons = [0, 1, 2, 5]
features = ['Longitude','PFG_surface_pressure','Latitude','Slope','Elevation','Year','EVI']
target = 'ALT_Max'

# Load and evaluate
for hrz in horizons:
    print(f'\n*********************** HRZ = {hrz} years')
    train_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])
    test_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])

    train_info = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv', usecols=['MainRegion', 'Region', 'Location', 'Latitude', 'Longitude', 'Year'])
    test_info = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv', usecols=['MainRegion', 'Region', 'Location', 'Latitude', 'Longitude', 'Year'])

    train_data = train_data[features + [target]]
    test_data = test_data[features + [target]]

    X_train = train_data.drop(columns=target)
    y_train = train_data[target]
    X_test = test_data.drop(columns=target)
    y_test = test_data[target]

    print("- Tuning Extra Trees...")
    et_model = ExtraTreesRegressor(random_state=42)
    et_params = {
        'n_estimators': [300, 500, 1000, 1500, 2000, 2500],
        'max_depth': [None, 5, 7, 10, 20, 30, 50],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4, 6],
        'max_features': ['sqrt', 'log2', 0.5, 0.7, 0.9]
    }
    et_search = RandomizedSearchCV(et_model, et_params, n_iter=500, cv=cv, scoring='neg_root_mean_squared_error', random_state=42, n_jobs=-1)
    et_search.fit(X_train, y_train)
    best_et = et_search.best_estimator_
    print(f"  Best ExtraTrees: {et_search.best_params_}")

    print("- Tuning Bagging Regressor...")
    bag_model = BaggingRegressor(random_state=42)
    bag_params = {
        'n_estimators': [300, 500, 1000, 1500, 2000, 2500],
        'max_samples': [0.6, 0.7, 0.8, 0.9, 1.0],
        'max_features': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'bootstrap_features': [True, False]
    }
    bag_search = RandomizedSearchCV(bag_model, bag_params, n_iter=500, cv=cv, scoring='neg_root_mean_squared_error', random_state=42, n_jobs=-1)
    bag_search.fit(X_train, y_train)
    best_bag = bag_search.best_estimator_
    print(f"  Best Bagging: {bag_search.best_params_}")

    # Define CatBoost model
    cat_model = CatBoostRegressor(
        loss_function='RMSE',
        random_seed=42,
        verbose=0
    )

    # Create training Pool
    train_pool = Pool(X_train, y_train)

    # Parameter search space
    param_grid = {
        'iterations': [300, 500, 1000, 1500, 2000, 2500],
        'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'bagging_temperature': [0, 1, 5],
        'random_strength': [1, 5, 10]
    }

    # Native grid search (CatBoost’s own)
    grid_result = cat_model.randomized_search(
        param_grid,
        X= X_train,
        y= y_train,
        cv=5,
        partition_random_seed=42,
        shuffle=True,
        refit=True,
        verbose=False,
        n_iter=500,
    )

    print(f"  Best Catboost: {grid_result['params']}")
    # Best CatBoost model after tuning
    best_cat = cat_model

    # Step 1: Define Models
    models = [
        ('Extra Trees', best_et),
        ('Bagging Regressor', best_bag),
        ('CatBoost',best_cat)
    ]

    preds_test = {}
    preds_train_cv = {}
    rmse_scores = {}

    # Step 2: CV predictions and RMSE
    for name, model in models:
        y_pred_cv = cross_val_predict(model, X_train, y_train, cv=cv, n_jobs=-1)
        model.fit(X_train, y_train)
        y_pred_test = model.predict(X_test)

        preds_test[name] = y_pred_test
        preds_train_cv[name] = y_pred_cv

        rmse = np.sqrt(mean_squared_error(y_train, y_pred_cv))
        r2 = r2_score(y_train, y_pred_cv)
        rmse_scores[name] = rmse

        print(f"{name}: CV Training RMSE = {rmse:.3f}, R²={r2:.3f} | Testing RMSE={np.sqrt(mean_squared_error(y_test, y_pred_test)):.3f}, R²={r2_score(y_test, y_pred_test):.3f}")

    # Step 3: Compute CV-based weights
    inverse_rmse = {k: 1 / v for k, v in rmse_scores.items()}
    total_inv = sum(inverse_rmse.values())
    weights = {k: v / total_inv for k, v in inverse_rmse.items()}

    # Step 4: Weighted predictions
    weighted_test_pred = sum(preds_test[m] * weights[m] for m in weights)
    weighted_cv_pred = sum(preds_train_cv[m] * weights[m] for m in weights)

    # Step 5: Evaluate weighted ensemble
    rmse = np.sqrt(mean_squared_error(y_test, weighted_test_pred))
    mae = mean_absolute_error(y_test, weighted_test_pred)
    r2 = r2_score(y_test, weighted_test_pred)
    print(f"==> Weighted Ensemble - RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

    # Step 6: Build outputs with metadata
    train_info = train_info.reset_index(drop=True)
    test_info = test_info.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    df_train_out = train_info.copy()
    df_test_out = test_info.copy()

    df_train_out['y_train'] = y_train
    df_test_out['y_test'] = y_test

    for name in preds_train_cv:
        df_train_out[f'y_trpred_{name}'] = preds_train_cv[name]
        df_test_out[f'y_tepred_{name}'] = preds_test[name]
        df_train_out[f'weight_{name}'] = weights[name]
        df_test_out[f'weight_{name}'] = weights[name]

    df_train_out['y_trpred_ensemble'] = weighted_cv_pred
    df_train_out['weight_ensemble'] = 1.0
    df_test_out['y_tepred_ensemble'] = weighted_test_pred
    df_test_out['weight_ensemble'] = 1.0

    # Save if needed
    df_train_out.to_csv(f'/content/drive/MyDrive/UND/ALT/ALT_hrz{hrz}_TrainPredictions.csv', index=False)
    df_test_out.to_csv(f'/content/drive/MyDrive/UND/ALT/ALT_hrz{hrz}_TestPredictions.csv', index=False)



*********************** HRZ = 0 years
- Tuning Extra Trees...


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


  Best ExtraTrees: {'n_estimators': 901, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': None}
- Tuning Bagging Regressor...
  Best Bagging: {'n_estimators': 2367, 'max_samples': 1.0, 'max_features': 1.0, 'bootstrap_features': False}

bestTest = 22.59740196
bestIteration = 102


bestTest = 20.91807541
bestIteration = 147


bestTest = 16.9647639
bestIteration = 207


bestTest = 13.19485159
bestIteration = 601


bestTest = 21.31507923
bestIteration = 775


bestTest = 15.76711526
bestIteration = 893


bestTest = 19.07715095
bestIteration = 928


bestTest = 15.50702748
bestIteration = 946


bestTest = 18.64587757
bestIteration = 1279


bestTest = 12.20937782
bestIteration = 1354


bestTest = 12.81696159
bestIteration = 2054


bestTest = 13.13668544
bestIteration = 2270


bestTest = 17.56680511
bestIteration = 2387


bestTest = 23.91630813
bestIteration = 66


bestTest = 20.8287443
bestIteration = 95


bestTest = 17.60866424
bestIteration = 556


bestTest = 

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


  Best ExtraTrees: {'n_estimators': 1594, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.7, 'max_depth': 50}
- Tuning Bagging Regressor...
  Best Bagging: {'n_estimators': 1877, 'max_samples': 1.0, 'max_features': 1.0, 'bootstrap_features': False}

bestTest = 23.2001564
bestIteration = 102


bestTest = 21.42712946
bestIteration = 147


bestTest = 16.7186077
bestIteration = 206


bestTest = 13.1717608
bestIteration = 601


bestTest = 21.65720572
bestIteration = 774


bestTest = 16.25556563
bestIteration = 893


bestTest = 19.46532265
bestIteration = 928


bestTest = 15.98142295
bestIteration = 946


bestTest = 19.35233104
bestIteration = 1279


bestTest = 12.72095229
bestIteration = 1353


bestTest = 13.12611301
bestIteration = 2053


bestTest = 13.52746714
bestIteration = 2270


bestTest = 18.27627766
bestIteration = 2387


bestTest = 24.31602181
bestIteration = 66


bestTest = 21.28784099
bestIteration = 95


bestTest = 18.18137917
bestIteration = 556


bestTest = 13

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


  Best ExtraTrees: {'n_estimators': 2061, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 20}
- Tuning Bagging Regressor...
  Best Bagging: {'n_estimators': 1733, 'max_samples': 1.0, 'max_features': 0.9, 'bootstrap_features': False}

bestTest = 22.42188916
bestIteration = 102


bestTest = 20.90022386
bestIteration = 147


bestTest = 17.12616507
bestIteration = 207


bestTest = 12.98268744
bestIteration = 601


bestTest = 21.18827387
bestIteration = 775


bestTest = 15.42703278
bestIteration = 893


bestTest = 19.16521673
bestIteration = 928


bestTest = 15.1199085
bestIteration = 946


bestTest = 18.39559318
bestIteration = 1279


bestTest = 12.3892123
bestIteration = 1353


bestTest = 12.32493473
bestIteration = 2054


bestTest = 12.78460107
bestIteration = 2270


bestTest = 16.92692052
bestIteration = 2386


bestTest = 24.70282828
bestIteration = 66


bestTest = 21.12547534
bestIteration = 95


bestTest = 17.41278421
bestIteration = 555


bestTest = 1

In [ ]:
# Main script for proposed model learning with tuning
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import RandomizedSearchCV

# Configuration
horizons = [0, 1, 2, 5]
features = ['Longitude','PFG_surface_pressure','Latitude','Slope','Elevation','Year','EVI']
target = 'ALT_Max'


hyperparms_ET = {0: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 20},
                 1: {'n_estimators': 700, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 50},
                 2: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 20},
                 5: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 0.9, 'max_depth': 20}}

hyperparms_Bagging = {0: {'n_estimators': 2500, 'max_samples': 1.0, 'max_features': 1.0, 'bootstrap_features': False},
                 1: {'n_estimators': 2500, 'max_samples': 1.0, 'max_features': 0.8, 'bootstrap_features': False},
                 2: {'n_estimators': 2000, 'max_samples': 1.0, 'max_features': 1.0, 'bootstrap_features': True},
                 5: {'n_estimators': 2500, 'max_samples': 1.0, 'max_features': 0.9, 'bootstrap_features': False}}

hyperparms_cat = {0: {'bagging_temperature': 5, 'random_strength': 1, 'depth': 6, 'learning_rate': 0.1, 'l2_leaf_reg': 7, 'iterations': 1500},
                 1: {'bagging_temperature': 1, 'random_strength': 1, 'depth': 4, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 2500},
                 2: {'bagging_temperature': 1, 'random_strength': 1, 'depth': 6, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 2500},
                 5: {'bagging_temperature': 1, 'random_strength': 1, 'depth': 6, 'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 2500}}

# Load and evaluate
for hrz in horizons:
    # Dynamically define models for the current horizon
    models = [
        ('Extra Trees', ExtraTreesRegressor(random_state=42, **hyperparms_ET[hrz])),
        ('Bagging Regressor', BaggingRegressor(random_state=42, **hyperparms_Bagging[hrz])),
        ('CatBoost', CatBoostRegressor(verbose=0, random_seed=42, **hyperparms_cat[hrz]))
    ]
    print(f'\n*********************** HRZ = {hrz} years')
    train_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])
    test_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])

    train_info = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv', usecols=['MainRegion', 'Region', 'Location', 'Latitude', 'Longitude', 'Year'])
    test_info = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv', usecols=['MainRegion', 'Region', 'Location', 'Latitude', 'Longitude', 'Year'])

    train_data = train_data[features + [target]]
    test_data = test_data[features + [target]]

    X_train = train_data.drop(columns=target)
    y_train = train_data[target]
    X_test = test_data.drop(columns=target)
    y_test = test_data[target]

    preds_test = {}
    preds_train_cv = {}
    rmse_scores = {}

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    # Step 2: CV predictions and RMSE
    for name, model in models:
        y_pred_cv = cross_val_predict(model, X_train, y_train, cv=cv, n_jobs=-1)
        model.fit(X_train, y_train)
        y_pred_test = model.predict(X_test)

        preds_test[name] = y_pred_test
        preds_train_cv[name] = y_pred_cv

        rmse = np.sqrt(mean_squared_error(y_train, y_pred_cv))
        r2 = r2_score(y_train, y_pred_cv)
        rmse_scores[name] = rmse

        print(f"{name}: CV Training RMSE = {rmse:.3f}, R²={r2:.3f} | Testing RMSE={np.sqrt(mean_squared_error(y_test, y_pred_test)):.3f}, R²={r2_score(y_test, y_pred_test):.3f}")

    # Step 3: Compute CV-based weights
    inverse_rmse = {k: 1 / v for k, v in rmse_scores.items()}
    total_inv = sum(inverse_rmse.values())
    weights = {k: v / total_inv for k, v in inverse_rmse.items()}

    # Step 4: Weighted predictions
    weighted_test_pred = sum(preds_test[m] * weights[m] for m in weights)
    weighted_cv_pred = sum(preds_train_cv[m] * weights[m] for m in weights)

    # Step 5: Evaluate weighted ensemble
    rmse = np.sqrt(mean_squared_error(y_test, weighted_test_pred))
    mae = mean_absolute_error(y_test, weighted_test_pred)
    r2 = r2_score(y_test, weighted_test_pred)
    print(f"==> Weighted Ensemble - RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

    # Step 6: Build outputs with metadata
    train_info = train_info.reset_index(drop=True)
    test_info = test_info.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    df_train_out = train_info.copy()
    df_test_out = test_info.copy()

    df_train_out['y_train'] = y_train
    df_test_out['y_test'] = y_test

    for name in preds_train_cv:
        df_train_out[f'y_trpred_{name}'] = preds_train_cv[name]
        df_test_out[f'y_tepred_{name}'] = preds_test[name]
        df_train_out[f'weight_{name}'] = weights[name]
        df_test_out[f'weight_{name}'] = weights[name]

    df_train_out['y_trpred_ensemble'] = weighted_cv_pred
    df_train_out['weight_ensemble'] = 1.0
    df_test_out['y_tepred_ensemble'] = weighted_test_pred
    df_test_out['weight_ensemble'] = 1.0

    # Save if needed
    df_train_out.to_csv(f'/content/drive/MyDrive/UND/ALT/ALT_hrz{hrz}_TrainPredictions.csv', index=False)
    df_test_out.to_csv(f'/content/drive/MyDrive/UND/ALT/ALT_hrz{hrz}_TestPredictions.csv', index=False)



*********************** HRZ = 0 years
Extra Trees: CV Training RMSE = 12.595, R²=0.909 | Testing RMSE=24.946, R²=0.802
Bagging Regressor: CV Training RMSE = 13.045, R²=0.902 | Testing RMSE=26.362, R²=0.779
CatBoost: CV Training RMSE = 13.268, R²=0.899 | Testing RMSE=25.493, R²=0.793
==> Weighted Ensemble - RMSE: 24.241, MAE: 14.627, R²: 0.813

*********************** HRZ = 1 years
Extra Trees: CV Training RMSE = 13.063, R²=0.902 | Testing RMSE=24.494, R²=0.809
Bagging Regressor: CV Training RMSE = 14.206, R²=0.884 | Testing RMSE=26.920, R²=0.770
CatBoost: CV Training RMSE = 13.703, R²=0.892 | Testing RMSE=26.730, R²=0.773
==> Weighted Ensemble - RMSE: 24.328, MAE: 14.653, R²: 0.812

*********************** HRZ = 2 years
Extra Trees: CV Training RMSE = 12.795, R²=0.906 | Testing RMSE=24.488, R²=0.809
Bagging Regressor: CV Training RMSE = 14.106, R²=0.886 | Testing RMSE=27.192, R²=0.765
CatBoost: CV Training RMSE = 13.242, R²=0.899 | Testing RMSE=24.779, R²=0.805
==> Weighted Ensemble -

## Benchmark against other models

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Sklearn models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# Horizons and config
horizons = [0, 1, 2, 5]
target = 'ALT_Max'

features = ['Longitude','PFG_surface_pressure','Latitude','Slope','Elevation','Year','EVI']

# Conventional ML models
sklearn_models = [
    ('Linear Regression', LinearRegression()),
    ('Ridge', Ridge(alpha=1.0)),
    ('Lasso', Lasso(alpha=0.01)),
    ('ElasticNet', ElasticNet(alpha=0.01, l1_ratio=0.5)),
    ('Decision Tree', DecisionTreeRegressor(random_state=42)),
    ('Random Forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('KNN', KNeighborsRegressor(n_neighbors=5)),
    ('SVR', SVR(kernel='rbf')),
    ('MLP', MLPRegressor(hidden_layer_sizes=(200, 100, 50, 25), max_iter=500, random_state=42))
]


# Main loop
for hrz in horizons:
    print(f'\n{"="*15} HRZ = {hrz} years {"="*15}')

    # Load data
    train_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])
    test_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])

    train_data = train_data[features + [target]]
    test_data = test_data[features + [target]]

    X_train, y_train = train_data.drop(columns=target), train_data[target]
    X_test, y_test = test_data.drop(columns=target), test_data[target]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    for name, model in sklearn_models:
        try:
            y_pred_cv = cross_val_predict(model, X_train_scaled, y_train, cv=cv)
            model.fit(X_train_scaled, y_train)
            y_pred_test = model.predict(X_test_scaled)

            rmse_cv = np.sqrt(mean_squared_error(y_train, y_pred_cv))
            mae_cv = mean_absolute_error(y_train, y_pred_cv)
            r2_cv = r2_score(y_train, y_pred_cv)
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
            mae_test = mean_absolute_error(y_test, y_pred_test)
            r2_test = r2_score(y_test, y_pred_test)

            print(f"{name}: RMSE = {rmse_cv:.3f}, MAE = {mae_cv:.3f}, R² = {r2_cv:.3f}, Test RMSE = {rmse_test:.3f}, MAE = {mae_test:.3f}, R² = {r2_test:.3f}")
        except Exception as e:
            print(f"{name}: FAILED - {str(e)}")


=============== HRZ = 0 years ===============
Linear Regression: RMSE = 38.789, MAE = 29.240, R² = 0.137, Test RMSE = 54.169, MAE = 38.822, R² = 0.068
Ridge: RMSE = 38.788, MAE = 29.238, R² = 0.137, Test RMSE = 54.170, MAE = 38.821, R² = 0.068
Lasso: RMSE = 38.788, MAE = 29.237, R² = 0.137, Test RMSE = 54.171, MAE = 38.821, R² = 0.068
ElasticNet: RMSE = 38.787, MAE = 29.229, R² = 0.137, Test RMSE = 54.177, MAE = 38.816, R² = 0.067
Decision Tree: RMSE = 15.972, MAE = 9.550, R² = 0.854, Test RMSE = 33.774, MAE = 19.029, R² = 0.638
Random Forest: RMSE = 13.200, MAE = 8.202, R² = 0.900, Test RMSE = 26.474, MAE = 14.963, R² = 0.777
KNN: RMSE = 19.637, MAE = 12.517, R² = 0.779, Test RMSE = 33.002, MAE = 20.572, R² = 0.654
SVR: RMSE = 35.857, MAE = 25.164, R² = 0.263, Test RMSE = 53.003, MAE = 34.557, R² = 0.107
MLP: RMSE = 19.118, MAE = 12.814, R² = 0.790, Test RMSE = 30.882, MAE = 21.789, R² = 0.697

=============== HRZ = 1 years ===============
Linear Regression: RMSE = 38.833, MAE = 29.2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras import layers, models, callbacks, Input, Model
from sklearn.model_selection import KFold
import tensorflow as tf
import os
import random

# Set seeds
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# === Model Architectures ===

import tensorflow as tf
from tensorflow.keras import layers, Model, Input

def bottleneck_block(x, filters, downsample=False):
    shortcut = x

    # 1x1 reduce
    x = layers.Dense(filters, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # 3x3 process (still Dense)
    x = layers.Dense(filters, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # 1x1 expand
    x = layers.Dense(filters * 4)(x)
    x = layers.BatchNormalization()(x)

    # Projection shortcut if needed
    if downsample or shortcut.shape[-1] != x.shape[-1]:
        shortcut = layers.Dense(filters * 4)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def build_resnet50_tabular(input_dim):
    inputs = Input(shape=(input_dim,))
    x = layers.Dense(64)(inputs)  # Initial projection
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Block layout: 3, 4, 6, 3 (as in ResNet-50)
    for reps, filters in zip([3, 4, 6, 3], [64, 128, 256, 512]):
        for i in range(reps):
            x = bottleneck_block(x, filters, downsample=(i == 0))

    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(1)(x)
    return Model(inputs, x)


def build_vgg16_tabular(input_dim):
    inputs = Input(shape=(input_dim,))
    x = inputs
    for units in [64, 64, 128, 128, 256, 256, 256, 512, 512, 512, 512, 512, 512]:
        x = layers.Dense(units, activation='relu')(x)
        x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    output = layers.Dense(1)(x)
    return Model(inputs, output)


def build_ft_transformer_continuous(input_dim, num_heads=8, num_blocks=4, dim=64,
                                    attn_dropout=0.1, ff_dropout=0.1):
    inputs = tf.keras.Input(shape=(input_dim,))

    # Project continuous input: [B, F] → [B, F, 1] → [B, F, dim]
    x = tf.keras.layers.Reshape((input_dim, 1))(inputs)
    tokens = tf.keras.layers.Dense(dim)(x)

    # Add CLS token
    cls_token = tf.Variable(tf.random.normal([1, 1, dim]), trainable=True, name='cls_token')
    cls_layer = tf.keras.layers.Lambda(lambda t: tf.concat([tf.tile(cls_token, [tf.shape(t)[0], 1, 1]), t], axis=1))
    tokens = cls_layer(tokens)  # [B, F+1, dim]

    # Positional (column) encodings
    pos_embeds = tf.keras.layers.Embedding(input_dim + 1, dim)
    def add_positional_encoding(x):
        seq_len = tf.shape(x)[1]
        pos_indices = tf.range(start=0, limit=seq_len, delta=1)
        pos_encoding = pos_embeds(pos_indices)
        return x + tf.expand_dims(pos_encoding, axis=0)
    tokens = tf.keras.layers.Lambda(add_positional_encoding)(tokens)

    x = tokens
    for _ in range(num_blocks):
        # Multi-head self-attention with dropout
        attn_input = tf.keras.layers.LayerNormalization()(x)
        attn_output = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=dim,
            dropout=attn_dropout
        )(attn_input, attn_input)
        attn_output = tf.keras.layers.Dropout(attn_dropout)(attn_output)
        x = tf.keras.layers.Add()([x, attn_output])

        # Feedforward MLP with dropout
        ff_input = tf.keras.layers.LayerNormalization()(x)
        ff = tf.keras.layers.Dense(dim * 2, activation='relu')(ff_input)
        ff = tf.keras.layers.Dropout(ff_dropout)(ff)
        ff = tf.keras.layers.Dense(dim)(ff)
        ff = tf.keras.layers.Dropout(ff_dropout)(ff)
        x = tf.keras.layers.Add()([x, ff])

    # Regress on the CLS token
    x = tf.keras.layers.Lambda(lambda x: x[:, 0])(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    output = tf.keras.layers.Dense(1)(x)
    return tf.keras.Model(inputs, output)



def build_densenet121_tabular(input_dim):
    inputs = Input(shape=(input_dim,))
    x = layers.Dense(64, activation='relu')(inputs)

    for num_layers in [6, 12, 24, 16]:
        for _ in range(num_layers):
            new_feat = layers.Dense(32, activation='relu')(x)
            x = layers.Concatenate()([x, new_feat])
        x = layers.Dense(64, activation='relu')(x)  # transition layer

    x = layers.Dense(128, activation='relu')(x)
    output = layers.Dense(1)(x)
    return Model(inputs, output)

def build_autoint_continuous(input_dim, embedding_dim=32, num_heads=4, num_layers=3, dropout_rate=0.1):
    inputs = tf.keras.Input(shape=(input_dim,))

    # Per-feature embedding: [B, F] → [B, F, E]
    x = tf.keras.layers.Reshape((input_dim, 1))(inputs)
    x = tf.keras.layers.Dense(embedding_dim)(x)

    for _ in range(num_layers):
        attn_input = tf.keras.layers.LayerNormalization()(x)  # Norm before attention
        attn_output = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embedding_dim, dropout=dropout_rate
        )(attn_input, attn_input)

        attn_output = tf.keras.layers.Dropout(dropout_rate)(attn_output)
        x = tf.keras.layers.Add()([x, attn_output])  # residual
        # No activation after residual per original AutoInt

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    output = tf.keras.layers.Dense(1)(x)
    return tf.keras.Model(inputs=inputs, outputs=output)

# === Model Registry ===
model_builders = {
    # "ResNet50_Tab": build_resnet50_tabular,
    # "VGG16_Tab": build_vgg16_tabular,
    # "DenseNet121_Tab": build_densenet121_tabular,
    # "FTTransformer": build_ft_transformer_continuous,
    "AutoInt": build_autoint_continuous,
    }#"MLP-Mixer": build_mlp_mixer

# === Loop Over Horizons ===

results = []
horizons = [0, 1, 2, 5]
target = 'ALT_Max'

features = ['Longitude','PFG_surface_pressure','Latitude','Slope','Elevation','Year','EVI']


for hrz in horizons:
    print(f'\n{"="*15} HRZ = {hrz} years {"="*15}')

    # Load data
    train_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])
    test_data = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TestDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])

    target = 'ALT_Max'

    train_data = train_data[features + [target]]
    test_data = test_data[features + [target]]

    X_train = train_data[features].values
    y_train = train_data[target].values
    X_test = test_data[features].values
    y_test = test_data[target].values

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    input_dim = X_train_scaled.shape[1]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    for model_name, build_fn in model_builders.items():
        print(f"\nModel: {model_name}")

        fold_rmse, fold_mae, fold_r2 = [], [], []

        for train_idx, val_idx in kf.split(X_train_scaled):
            X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]

            model = build_fn(X_tr.shape[1])
            model.compile(optimizer='adam', loss='mse', metrics=['mae'])

            early_stop = callbacks.EarlyStopping(patience=100, restore_best_weights=True, verbose=0)
            model.fit(X_train_scaled, y_train, epochs=20000, batch_size=64,
                      validation_split=0.2, verbose=0, callbacks=[early_stop])

            y_pred = model.predict(X_val).flatten()
            fold_rmse.append(np.sqrt(mean_squared_error(y_val, y_pred)))
            fold_mae.append(mean_absolute_error(y_val, y_pred))
            fold_r2.append(r2_score(y_val, y_pred))

        model_full = build_fn(X_train_scaled.shape[1])
        model_full.compile(optimizer='adam', loss='mse', metrics=['mae'])
        model_full.fit(X_train_scaled, y_train, epochs=20000, batch_size=64,
                      validation_split=0.2, verbose=0, callbacks=[early_stop])
        y_pred_test = model_full.predict(X_test_scaled).flatten()
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        mae = mean_absolute_error(y_test, y_pred_test)
        r2 = r2_score(y_test, y_pred_test)
        print(f'{model_name}, RMSE={np.mean(fold_rmse)}, MAE={np.mean(fold_mae)}, R2={np.mean(fold_r2)}, RMSE={rmse}, MAE={mae}, R2={r2}')

        results.append({
            "Horizon": hrz,
            "Model": model_name,
            "trRMSE": np.mean(fold_rmse),
            "trMAE": np.mean(fold_mae),
            "trR2": np.mean(fold_r2),
            "RMSE": rmse,
            "MAE": mae,
            "R2": r2
        })

# === Display Results ===

results_df = pd.DataFrame(results)
print("\nFinal Results:")
print(results_df)

# Optional: Save to CSV
# results_df.to_csv("ALT_DeepModelSweep_Keras.csv", index=False)


## Stats on the selected features

In [ ]:
!pip install xicorpy pandas numpy scipy statsmodels

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
import xicorpy

# Parameters
target = 'ALT_Max'
features = ['Longitude', 'PFG_surface_pressure', 'Latitude', 'Slope', 'Elevation', 'Year', 'EVI']
horizons = [0, 1, 2, 5]

# Initialize dictionary to collect all results
results_dict = {feature: {} for feature in features}

for hrz in horizons:
    # Load and prepare data
    df = pd.read_csv(f'/content/drive/MyDrive/UND/ALT/TrainDatawERA5_{hrz}Years_V3.csv').drop(columns=['MainRegion', 'Region', 'Location'])
    df = df[features + [target]].dropna()

    # Standardize for VIF
    X_scaled = StandardScaler().fit_transform(df[features])
    vif_list = [variance_inflation_factor(X_scaled, i) for i in range(len(features))]
    vif_dict = dict(zip(features, vif_list))

    for feature in features:
        x = df[feature].values
        y = df[target].values
        xi_corr, _ = xicorpy.compute_xi_correlation(x, y, get_p_values=True)
        spear_corr, _ = spearmanr(x, y)

        results_dict[feature][f'Xi h={hrz}'] = xi_corr[0][0]
        # results_dict[feature][f'R h={hrz}'] = spear_corr
        results_dict[feature][f'VIF h={hrz}'] = vif_dict[feature]

# Build DataFrame
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

# Reset index to get Feature as a column
results_df = results_df.reset_index().rename(columns={'index': 'Feature'})
results_df

,Feature,Xi h=0,VIF h=0,Xi h=1,VIF h=1,Xi h=2,VIF h=2,Xi h=5,VIF h=5
0,Longitude,0.877587,1.293832,0.877668,1.292808,0.877712,1.294540,0.877712,1.317113
1,PFG_surface_pressure,0.292654,1.182320,0.277294,1.169722,0.272236,1.165739,0.292375,1.156887
2,Latitude,0.863479,1.479798,0.863570,1.471344,0.863611,1.479566,0.863611,1.491133
3,Slope,0.879904,1.344762,0.879987,1.344362,0.880026,1.343154,0.880026,1.335490
4,Elevation,0.873629,1.477361,0.873711,1.477603,0.873754,1.480348,0.873754,1.488870
5,Year,0.317010,1.576118,0.317540,1.638584,0.317708,1.687120,0.317708,1.850111
6,EVI,0.247202,1.753994,0.248141,1.791768,0.276092,1.839827,0.337036,2.001901
